In [1]:
from ortools.graph import pywrapgraph
import pandas as pd
import numpy as np

from functions import *

In [2]:
#https://developers.google.com/optimization/flow/mincostflow
#https://towardsdatascience.com/operations-research-in-r-transportation-problem-1df59961b2ad

def solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies ):
    """MinCostFlow simple interface example."""
    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each arc.
    for arc in zip(start_nodes, end_nodes, capacities, unit_costs):
        min_cost_flow.AddArcWithCapacityAndUnitCost(arc[0], arc[1], arc[2],
                                                    arc[3])

    # Add node supply.
    for count, supply in enumerate(supplies):
        min_cost_flow.SetNodeSupply(count, supply)

    # Find the min cost flow.
    status = min_cost_flow.Solve()

    print("Solving Problem")

    if status != min_cost_flow.OPTIMAL:
        print('There was an issue with the min cost flow input.')
        print(f'Status: {status}')
        if status == min_cost_flow.INFEASIBLE:
            print("The problem is infeasible")
        if status == min_cost_flow.UNBALANCED:
            print("The problem is unbalanced")


    print('Minimum cost: ', min_cost_flow.OptimalCost())
    print('')
    print(' Arc   Flow / Capacity  Cost')
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        print('%1s -> %1s    %3s   / %3s   %3s' %
              (min_cost_flow.Tail(i), min_cost_flow.Head(i),
               min_cost_flow.Flow(i), min_cost_flow.Capacity(i), cost))
    

In [3]:
#pd.read_excel
df_cap = pd.read_excel('docs/io_v2.xlsx', 
            sheet_name = 'Capacities', 
            usecols = [0,1,2], 
             dtype = {
                        'NodeIndex': str, 
                        'NodeName': str, 
                        'Capacity': float}).dropna()

# Load Nodes connections
df_conn = pd.read_excel('docs/io_v2.xlsx', 
                    sheet_name = 'Connections', usecols = [0,1,2,3], 
                    dtype = {
                        'StartNode': str, 
                        'EndNode': str, 
                        'Distance': float,
                        'Cost': float}).dropna()

# Supplier to get out
prov = ['Prov 6']

# Eliminate Supplier 6
df_cap = df_cap[~df_cap['NodeName'].isin(prov)].copy(deep = True).reset_index(drop = True)
df_conn = df_conn[~((df_conn['StartNode'].isin(prov)) | (df_conn['EndNode'].isin(prov)))].copy(deep = True).reset_index(drop = True)

cap_prov = df_cap.loc[df_cap['NodeName'].apply(lambda x: 'Prov' in x)]['Capacity'].sum()
cap_client = df_cap.loc[df_cap['NodeName'].apply(lambda x: 'Cliente' in x)]['Capacity'].sum()

# Check for imbalance problem and adjust it
df_cap, df_conn = adjust_umbalance_problem(df_cap, df_conn, cap_prov, cap_client )

# Reindex Nodes
df_cap['NodeIndex'] = df_cap.index

# Drop na's
df_cap.dropna(inplace = True)

# Adjust Capcity sign
df_cap['sign'] = df_cap['NodeName'].apply(lambda x: -1 if 'Cliente' in x else 1)
df_cap['Capacity'] = df_cap['Capacity'] * df_cap['sign']


# Trim Start and EndNode
cols = ['StartNode', 'EndNode']
df_conn = strip_col(df_conn, cols)

# Trim Node name
cols = ['NodeName']
df_cap = strip_col(df_cap, cols)

# Adjust node indeces in the Connections data frame
node_mapper = dict(zip(df_cap['NodeName'], df_cap['NodeIndex'].astype(int)))
df_conn['StartNodeIndex'] = df_conn['StartNode'].apply(lambda x: int(node_mapper.get(x)) )
df_conn['EndNodeIndex'] = df_conn['EndNode'].apply(lambda x: int(node_mapper.get(x)) )
df_conn['Total_Cost'] = (df_conn['Distance'] * df_conn['Cost'] * 10).astype(int)

 Problem is imbalanced, the supply is higher than the demand


In [5]:
# Create Input Vectors
c = 10**5

start_nodes = df_conn['StartNodeIndex'].to_list()
end_nodes = df_conn['EndNodeIndex'].to_list()
capacities = [c] * df_conn.shape[0]
unit_costs = df_conn['Total_Cost'].astype(int).to_list() 

# Define an array of supplies at each node.
supplies = df_cap['Capacity'].astype(int).to_list()

In [7]:
node_mapper

{'Prov 1': 0,
 'Prov 2': 1,
 'Prov 3': 2,
 'Prov 4': 3,
 'Prov 5': 4,
 'Cliente 1': 5,
 'Cliente 2': 6,
 'Cliente 3': 7,
 'Cliente 4': 8,
 'Cliente 5': 9,
 'Cliente 6': 10,
 'Cliente 7': 11,
 'Cliente 8': 12,
 'Cliente 9': 13,
 'Cliente 10': 14,
 'Cliente 11': 15,
 'Cliente 12': 16,
 'Cliente 13': 17,
 'Cliente 14': 18,
 'Cliente 15': 19,
 'Cliente 16': 20,
 'Cliente 17': 21,
 'Cliente 18': 22,
 'Cliente 19': 23,
 'Cliente 20': 24,
 'Cliente 21': 25,
 'Cliente 22': 26,
 'Cliente 23': 27,
 'Cliente 24': 28,
 'Cliente 25': 29,
 'Cliente Add': 30}

In [6]:
solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies)

Solving Problem
Minimum cost:  55195

 Arc   Flow / Capacity  Cost
0 -> 5      0   / 100000     0
0 -> 6      1   / 100000    42
0 -> 7      0   / 100000     0
0 -> 8     44   / 100000   572
0 -> 9     39   / 100000   1131
0 -> 10     63   / 100000   2268
0 -> 11      0   / 100000     0
0 -> 12      0   / 100000     0
0 -> 13      0   / 100000     0
0 -> 14      0   / 100000     0
0 -> 15      0   / 100000     0
0 -> 16      0   / 100000     0
0 -> 17     30   / 100000   1200
0 -> 18      0   / 100000     0
0 -> 19      0   / 100000     0
0 -> 20      0   / 100000     0
0 -> 21      0   / 100000     0
0 -> 22      0   / 100000     0
0 -> 23      0   / 100000     0
0 -> 24    110   / 100000   4620
0 -> 25      0   / 100000     0
0 -> 26      0   / 100000     0
0 -> 27    143   / 100000   4433
0 -> 28      0   / 100000     0
0 -> 29      0   / 100000     0
1 -> 5     40   / 100000   1840
1 -> 6     36   / 100000   1404
1 -> 7    115   / 100000   2875
1 -> 8      0   / 100000     0
1 -> 9